In [46]:
# Загрузка документов
import os
from langchain_community.document_loaders import WebBaseLoader


os.environ["USER_AGENT"] = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"

loader = WebBaseLoader('https://antarcticwallet.com/faq')
docs = loader.load()

print(f"Total pages loaded: {len(docs)}")
print(f"Total characters: {sum(len(doc.page_content) for doc in docs)}")

Total pages loaded: 1
Total characters: 10004


In [17]:
# Разделение документов на кусочки (split)
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [19]:
print(splits[0].page_content.replace('\n', ' '))
print("----------------------")
print(splits[1].page_content.replace('\n', ' '))
print('----------------------')
print(splits[2].page_content.replace('\n', ' '))

Вопросы и ответы – Antarctic Wallet                                                                             Открыть кошелёк       Меню       Функционал       FAQ       Контакты       Безопасность                                                                                                                                     Открыть кошелёк       Закрыть     Кошелёк   Компания   Медиа     Telegram канал       YouTube       Нельзяграмм       Блог     О компании   Контакты   Лицензия   Политика AML   Для официальных запросов   Процесс оплаты   Реферальная программа   Техподдержка   FAQ   Как установить   Продажа USDT на карту   Весь функционал               Switch to English                                                                                                                                  AppStore - soon       Web-версия       Telegram Bot     Где вы хотите открыть Antarctic Wallet?                                                                          База
----------

In [20]:
# Создаём embeddings
from langchain_ollama import OllamaEmbeddings

# https://huggingface.co/Casual-Autopsy/snowflake-arctic-embed-l-v2.0-gguf
EMBEDDING_MODEL = 'hf.co/Casual-Autopsy/snowflake-arctic-embed-l-v2.0-gguf:Q4_K_M'
OLLAMA_EMBEDDING_BASE_URL = 'http://127.0.0.1:11434'

embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,      # имя модели в ollama
    base_url=OLLAMA_EMBEDDING_BASE_URL,   # стандартный адрес ollama
)
embeddings

OllamaEmbeddings(model='hf.co/Casual-Autopsy/snowflake-arctic-embed-l-v2.0-gguf:Q4_K_M', validate_model_on_init=False, base_url='http://127.0.0.1:11434', client_kwargs={}, async_client_kwargs={}, sync_client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [26]:
# Примеры запросов

vec1 = embeddings.embed_query("футболка")
vec2 = embeddings.embed_query("шорты")
vec3 = embeddings.embed_query("компьютер")
print(len(vec1), vec1[:5])
print(len(vec2), vec2[:5])
print(len(vec3), vec3[:5])

1024 [-0.04141563, 0.029370636, -0.020483648, 0.0067812596, -0.047986627]
1024 [-0.021739565, -0.004556779, 0.008548141, 0.00041586568, -0.011136292]
1024 [0.051323563, 0.02275423, 0.023706289, -0.027426371, -0.0090337405]


In [27]:
from langchain_community.vectorstores import Chroma


persist_directory = "./chroma_db"

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=persist_directory,
)

vectorstore

In [28]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
retriever

VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x1117994d0>, search_kwargs={'k': 5})

In [30]:
# Системный промпт
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "Ты помощник, который ОТВЕЧАЕТ СТРОГО НА РУССКОМ ЯЗЫКЕ. "
        "Даже если контекст или вопрос частично на других языках (английский, немецкий и т.п.), "
        "ты ВСЁ РАВНО отвечаешь только на русском, используя кириллицу. "
        "Не используй слова и фразы на других языках, кроме общепринятых имен собственных и названий. "
        "Используй следующие фрагменты контекста для ответа на вопрос. "
        "Если ты не знаешь ответа, просто скажи, что не знаешь. "
        "Используй максимум три предложения и будь лаконичным."
    ),
    (
        "human",
        "Контекст: {context}\n\nВопрос: {question}"
    ),
])

prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Ты помощник, который ОТВЕЧАЕТ СТРОГО НА РУССКОМ ЯЗЫКЕ. Даже если контекст или вопрос частично на других языках (английский, немецкий и т.п.), ты ВСЁ РАВНО отвечаешь только на русском, используя кириллицу. Не используй слова и фразы на других языках, кроме общепринятых имен собственных и названий. Используй следующие фрагменты контекста для ответа на вопрос. Если ты не знаешь ответа, просто скажи, что не знаешь. Используй максимум три предложения и будь лаконичным.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Контекст: {context}\n\nВопрос: {question}'), additional_kwargs={})])

In [8]:
# Будем использовать модль с поддержкой Русского языка:
# https://huggingface.co/bartowski/Mistral-Nemo-Instruct-2407-GGUF
# ollama pull hf.co/bartowski/Mistral-Nemo-Instruct-2407-GGUF:Q4_K_M

# если железо начинает лагать, возьмите модель попроще. Например: gemma3:4b
# ollama pull gemma3:4b

from langchain_openai import ChatOpenAI

OLLAMA_OPENAI_BASE_URL = 'http://127.0.0.1:11434/v1'
LLM_MODEL = 'hf.co/bartowski/Mistral-Nemo-Instruct-2407-GGUF:Q4_K_M'
# LLM_MODEL = 'gemma3:4b'

llm = ChatOpenAI(
    base_url=OLLAMA_OPENAI_BASE_URL,
    model=LLM_MODEL
)

llm

ChatOpenAI(profile={}, client=<openai.resources.chat.completions.completions.Completions object at 0x15913f8d0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x110f70450>, root_client=<openai.OpenAI object at 0x110f63d50>, root_async_client=<openai.AsyncOpenAI object at 0x110f616d0>, model_name='hf.co/bartowski/Mistral-Nemo-Instruct-2407-GGUF:Q4_K_M', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='http://127.0.0.1:11434/v1')

In [32]:
llm.invoke('Что такое Antarctic Wallet?')

AIMessage(content='Antarctic Wallet — это криптовалютный кошелёк, разработанный компанией Antarctic Ministries & Research. Antarctic Wallet известен своей поддержкой широкого спектра криптовалют и простотой в использовании.\n\nКак и многие другие кошельки, Antarctic Wallet хранит приватные ключи ваших криптовалютных адресов, что позволяет вам отправлять и получать криптовалюту. Он также имеет функцию обмена криптовалют прямо из кошелька без необходимости выхода на биржу.\n\nОцените статью', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 9, 'total_tokens': 140, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'hf.co/bartowski/Mistral-Nemo-Instruct-2407-GGUF:Q4_K_M', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-406', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2c86-96fb-7870-a527-5da9084e988d-0', usage_metadata={'input_tokens': 9, 'output_t

In [33]:
llm.invoke('Как оплатить криптовалютой по СБП QR-коду через Antarctic Wallet?')

AIMessage(content='Чтобы оплатить криптовалютой по Системе быстрых платежей (СБП) QR-кодом через кошелек Antarctic Wallet, вам нужно выполнить следующие шаги:\n\n1. Убедитесь, что у вас установлено приложение Antarctic Wallet и вы авторизованы в нем.\n2. Переходим к разделу "Оплата" или "QR-коды" в приложении.\n3. Выбираем опцию "СБП QR-код".\n4. Затем сканируйте generated QR код с помощью своего мобильного телефона для получения информации о получателе платежа.\n5. Выберите криптовалюту, которую вы хотите использовать для оплаты, из списка доступных в кошельке.\n6. Введите сумму платежа в соответствующем поле или используйте функцию "Сканировать" для обнаружения суммы из QR-кода.\n7. Подтвердите детали платежа и завершите транзакцию.\n\nПосле того, как вы завершили эти шаги, система СБП произведет обмен вашей криптовалюты на рубли или другую поддерживаемую валюту, а затем переведет средства получателю. Обратите внимание на комиссии и лимиты транзакций, применяющиеся к этой операции в 

In [35]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [36]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x1117994d0>, search_kwargs={'k': 5})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])
| ChatOpenAI(profile={},

In [39]:
# Question
rag_chain.invoke("Что такое Antarctic Wallet?")

'Антарктик Уоллет — это кошелек в Telegram для оплаты по QR-коду и обмена криптовалюты на карту. Он предоставляет функции платежей, безопасность и поддержку клиентов. Компания зарегистрирована в Бишкеке, Киргизия.'

In [40]:
# Question
rag_chain.invoke("Как начать пользвоваться Antarctic Wallet?")

'Чтобы начать пользоваться Antarctic Wallet, скачайте приложение через PWA на телефоне и авторизуйтесь через telegram-бота (@antarctic_wallet_bot).'

In [43]:
# Question
res = rag_chain.invoke("Какие комиссии есть в Antarctic Wallet?")
print(res)

Commission in Antarctic Wallet: USDT TRC20 network - $2.75, TON network - 0.2 TON, USDT TON network - $0.


In [44]:
# Question
rag_chain.invoke("Нужно ли проходить KYC?")

'Нужно ли проходить KYC в Antarctic Wallet?                 Да, для полного функционала кошелька необходимо пройти процедуру идентификации (KYC). Это связано с антиотмывочными мерами и Законом о противодействии финансированию терроризма.               При первом использовании некоторых функций, таких как продажа криптовалюты на карту, кошелек может потребовать верификацию личности.                  Если у вас есть конкретные вопросы по процедуре KYC, пожалуйста, обратитесь в службу поддержки или изучите Политику AML и FAQ раздел на официальном сайте или Telegram канале.'

In [45]:
# Question
rag_chain.invoke("Назови часы работы технической поддержки")

'Техническая поддержка Antarctic Wallet доступна в режиме онлайн, но время ответа может варьироваться. Если вы не получили ответ в течение 2-х часов, пожалуйста, продублируйте ваше сообщение. Для быстрой связи с нами, следуйте инструкциям на странице «Контакты» в разделе FAQ или обратитесь напрямую к боту технической поддержки по адресу @Antarcticwalletsupport_bot в Telegram.\nЧасы работы службы поддержки не указаны в предоставленном контексте.'